# DILATION IN POINT CLOUDS

This is a short tutorial to explain step by step morphological dilation applied to point clouds according to the work:

- Balado, J., Van Oosterom, P., Díaz-Vilariño, L., & Meijers, M. (2020). Mathematical morphology directly applied to point cloud data. ISPRS Journal of Photogrammetry and Remote Sensing, 168, 208-220.


# Import of libraries

The libraries used in the mathematical morphology are *numpy* and *o3d*.

- https://numpy.org/
- http://www.open3d.org/

In [ ]:
import numpy as np
import open3d as o3d

# Reading data

Although there are many point cloud formats, in this tutorial we will start from *txt* clouds, which can be read using *numpy*, as they do not have any kind of compression. The *txt* cloud is structured in 1 point per row and 1 attribute per column, and ' ' is specified as delimiter between columns. In this case, the mathematical morphology only works on XYZ data. 

As a working example, a cloud in open cube format is provided.


In [ ]:
# Read point cloud
input_data = np.loadtxt("Nubes/cubo3d.txt", delimiter=' ')

# Visualisation of the input point cloud

To visualise the input data in 3D, we use the *o3d* library, for which it is necessary to first transform our point cloud to a point cloud object of the library:

* Note: the predefined axes of the visualisation do not correspond to the real axes of the point cloud.

* Note: For those unfamiliar with accessing point cloud data, both in *numpy* and in other libraries, access is via [n rows, n columns], in such a way that ":" indicates that all the rows or columns are selected, and numerically "n:m" indicates that access is from row/column "n+1" to m.

In [ ]:
# Creat point cloud object
pcd_in = o3d.geometry.PointCloud()

# Load points into the object, delimited to XYZ in case the cloud has more attributes
pcd_in.points = o3d.utility.Vector3dVector(input_data[:,0:3])

# Visualization
o3d.visualization.draw_geometries([pcd_in])

# Define Structuring Element (SE)

The structuring element (SE) is a point cloud that we want to combine with the input cloud in order to modify it, either by dilation or erosion operations. Therefore, the SE contains only XYZ coordinates. The first point of the SE defines the centre of translation of the SE along the input point cloud. The rest of the points indicate the direction of dilation or erosion. Defining the SE correctly is the major complication of mathematical morphology and involves a process of trial and error.

<center> <img src="Figures/F01.jpg"></center>
<center> Figura 1. SE Examples </center>

In this example we define an SE of three points 0.1 metres apart in the Z direction and centred at [0 0 0], Figure 1.b.

In [ ]:
# SE definition
SE =  np.array([[0, 0, 0],
      [0, 0, -0.1],
      [0, 0, 0.1]])

# Define search distances

Unlike an image, whose pixel distribution is regular, in point clouds the distance between points varies, so it is necessary to indicate a search radius of points to establish proximities. 

In the original work, the search distance was automatically calculated in the process as the average of the distance between neighbouring points. However, defining the distance manually gives more freedom for object detection, segmentation and gap-filling operations. 

The distance between points of the SE, or the distance between points of the object to be dilated/erosionised, is proposed as the distance to be indicated.


In [ ]:
# Define search radius
d = 0.1

# Dilate

The dilation process is, broadly speaking, a process of adding new points to the cloud by following the steps below:

1. translate the SE to each point in the cloud. The translation of the SE is done by taking the first point of the SE as a reference.
2. Check if for each point of the SE there is a nearby point of the input cloud. The proximity calculation is done in *o3d* library to calculate distances between clouds and *d* is used to identify those distant points.
3. SE points that do not coincide with points in the input cloud are added to the input cloud (forming the dilated point cloud).

<center> <img src="Figures/F02.jpg"></center>
<center> Figura 2. Dilation process </center>

Two modifications were made to the original work to optimise the algorithm computationally. 
- The points are moved (iterations) over the SE and not over the input cloud, so fewer loops are performed. In this case, instead of moving the SE to each point in the input cloud, one point of the SE is moved to all points in the input cloud. 
- In the original algorithm, a density reduction was applied to eliminate repeated dilated points at the end of the loop. In this version, the distance check is performed on the set of input cloud + current dilated points, therefore, no previously dilated points within the loop influence the addition of new points.


In [ ]:
dilated_data = input_data[:,0:3]

for i in range(1,SE.shape[0]):
    
    # Move point i from the SE to the whole input cloud
    traslated_SE = input_data[:,0:3] + SE[i,:]
    
    # Convert to SE traslated to cloud-object
    pcd_tSE = o3d.geometry.PointCloud()
    pcd_tSE.points = o3d.utility.Vector3dVector(traslated_SE)
    
    # Convert concatenated point cloud to cloud-object
    pcd_dil = o3d.geometry.PointCloud()
    pcd_dil.points = o3d.utility.Vector3dVector(dilated_data)
    
    # Calculate distances between clouds 
    dist_pcd_tSE_2_pcd_dil = pcd_tSE.compute_point_cloud_distance(pcd_dil)
    dist_pcd_tSE_2_pcd_dil = np.asarray(dist_pcd_tSE_2_pcd_dil) 
    
    # Checking the existence of nearby points
    idx_add = dist_pcd_tSE_2_pcd_dil > d/2
    
    # Adding new points to the dilated cloud
    dilated_data = np.vstack((dilated_data,traslated_SE[idx_add,0:3]))
        

# Visualisation of the dilated point cloud

To visualise the dilated cloud in 3D, we use the *o3d* library, for which it is necessary to transform our output points back to a point cloud object from the *o3d* library:

* Note: the predefined axes of the visualisation do not correspond to the actual axes of the cloud.




In [ ]:
# Create point cloud object
pcd_out = o3d.geometry.PointCloud()

# Load points into the object
pcd_out.points = o3d.utility.Vector3dVector(dilated_data)

# Visualization
o3d.visualization.draw_geometries([pcd_out])


As can be seen, new points were generated according to the direction and shape of the SE in the input cloud.

# Export dilated point cloud

To save the cloud to disk, the *numpy* library and a *txt* cloud format are used. For saving, an address and name of the file to be generated are specified.

In [ ]:
# Definition of the path and file name
ruta = "Nubes/pc_dilated.txt"

# Save
np.savetxt(ruta,dilated_data,delimiter=' ') 

This concludes the morphological dilation tutorial. The application of morphological dilation combined with morphological erosion in morphological opening (for object segmentation) and morphological closing (for gap filling) operations will be presented in the next tutorial.